In [4]:
import pandas as pd 
import numpy as np

# DT 문제 1번

#### ※함수에 들어가 있는 변수나 flow는 본인이 바꾸셔도 가능하며 결과만 똑같이 나오면 됩니다!
#### ※hard코딩(이 데이터셋에만 적용되는 코딩방법) 말고 전체 데이터에 적용 가능하게 함수를 짜주셔야 합니다.

# Data Loading

In [5]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수에 대한 함수 만들기.

- Input은 Dataframe과 label이름으로 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다!

In [18]:
def get_gini(df, label):
    
    # label에 따른 분포 정보
    pair = df[label].value_counts()
    print(pair)
    feature_value = pair.values
    print(feature_value)
    # 분모
    under = feature_value.sum()
    print(under)
    
    
    gini = 1
    for i in feature_value:
        gini -=(i/under)*(i/under) 
    return gini

In [21]:
get_gini(pd_data, 'age')

youth          5
senior         5
middle_aged    4
Name: age, dtype: int64
[5 5 4]
14


0.663265306122449

- 저 위에 부분 집합 중 우리가 원하는 집합(이진 분류)만 골라 내야하겠죠?
- 그 함수를 get_binary_split로 완성해주세요!
- 완성된 내용은 다음과 같이 나와야 합니다.

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 함수를 사용하여 만들어주세요!
- DataFrame의 index를 사용하여 만들면 굉장히 편합니다..! 예시를 아래에서 보여드릴게요.
- 결과는 아래 아래 아래 줄과 같아야 합니다.

In [10]:
def get_attribute_gini_index(df, attribute, label):
    
    #라벨 정보 
    label_class = df[label].value_counts()
    label_class = label_class.index
    
    # feature에 대한 정보 - youth,middle,old
    feature_class = df[attribute].value_counts()
    feature_class_vlaues = feature_class.values
    feature_classes = feature_class.index
    
    under = feature_class_vlaues.sum()
    sub_gini=[]
    
    for idx, cls in enumerate(feature_classes):
        data_A = df[df[attribute]==cls].index
        gini_A = get_gini(df.loc[data_A],label)
        
        data_B = df[df[attribute]!=cls].index
        gini_B = get_gini(df.loc[data_B],label)
        
        gini_val = ((feature_class_vlaues[idx]/under) * gini_A) + (((under-feature_class_vlaues[idx])/under) * gini_B)
        
        sub_gini.append(gini_val)
    
    
    
    result = dict(zip(feature_classes,sub_gini))
    return result

In [27]:
KK = pd_data[pd_data['age']=='youth'].index
KK

Int64Index([0, 1, 7, 8, 10], dtype='int64')

In [26]:
xx = pd_data[pd_data['age']=='senior'].index
xx

Int64Index([3, 4, 5, 9, 13], dtype='int64')

In [30]:
pd_data.loc[KK.union(xx)]

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes
10,youth,medium,yes,excellent,yes
13,senior,medium,no,excellent,no


In [11]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

no     3
yes    2
Name: class_buys_computer, dtype: int64
[3 2]
yes    7
no     2
Name: class_buys_computer, dtype: int64
[7 2]
yes    3
no     2
Name: class_buys_computer, dtype: int64
[3 2]
yes    6
no     3
Name: class_buys_computer, dtype: int64
[6 3]
yes    4
Name: class_buys_computer, dtype: int64
[4]
no     5
yes    5
Name: class_buys_computer, dtype: int64
[5 5]


{'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth': 0.3936507936507937}

- 여기서 가장 작은값으로 분류를 해야겠죠?

In [83]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) income 의 이진분류를 얻는 함수 get_binary_split(pd_data, "income")을 통해 보여주세요.

In [ ]:
'''그거 안썻음...'''

## 문제2) 가장 Gini계수가 높은 Feature 즉 분류를 하는데 가장 중요한 변수를 선정하시고 get_attribute_gini_index함수를 통해 Gini index를 제시해주세요.

In [98]:
for attr in pd_data.columns[:-1]:
    print(attr,' : ',min(get_attribute_gini_index(pd_data, attr, "class_buys_computer").items()))
    
'''age의 gini index = 0.3571로 가장 작다'''    

age  :  ('middle_aged', 0.35714285714285715)
income  :  ('high', 0.4428571428571429)
student  :  ('no', 0.3673469387755103)
credit_rating  :  ('excellent', 0.42857142857142855)


'age의 gini index = 0.3571로 가장 작다'

## 문제3) 2에서 구한 Feature로 DataFrame을 분류 해주시고 나눠진 2개의 클래스에서 각각 다음으로 중요한 Feature를 선정해주시고 Gini index를 제시해주세요.

In [95]:
'''먼저 age - middle vs youth,senior'''
data_2_left=pd_data[pd_data['age']=='middle_aged']
data_2_right=pd_data[pd_data['age']!='middle_aged']

In [96]:
data_2_left

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [97]:
data_2_right

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes
10,youth,medium,yes,excellent,yes
13,senior,medium,no,excellent,no


In [104]:
'''그 다음 split point가 되는 피쳐를 찾자 '''
for attr in data_2_right.columns[:-1]:
    print(attr,' : ',min(get_attribute_gini_index(data_2_right, attr, "class_buys_computer").items()))
    
'''student의 gini index = 0.3199로 가장 작다'''

age  :  ('senior', 0.48)
income  :  ('high', 0.375)
student  :  ('no', 0.31999999999999984)
credit_rating  :  ('excellent', 0.4166666666666667)


'student의 gini index = 0.3199로 가장 작다'